# Learning the Quantum Teleportation Protocol

<em> Copyright (c) 2021 Institute for Quantum Computing, Baidu Inc. All Rights Reserved. </em>

## Overview

Quantum teleportation is another important task that can be completed by Local Operations and Classical Communication (LOCC) protocols, which transfers quantum information between two spatially separated communication nodes (only classical communication channel allowed) with the help of entanglement. In this tutorial, we will first briefly review the original teleportation protocol and simulate it with Paddle Quantum. Then, we will go through how to learn a teleportation protocol with LOCCNet.
 
## The original quantum teleportation protocol


The original teleportation protocol was proposed by C. H. Bennett et al. in 1993 [1] and experimentally verified in 1997 with photonic platforms [2-3]. The workflow is illustrated in the figure below. Following the convention, this process requires 2 communication nodes or parties, namely $A$ (Alice) and $B$ (Bob). For simplicity, we only consider transferring a single-qubit quantum state $|\psi\rangle_C$ and this requires 3 qubits in total including the pre-shared maximally entangled state $|\Phi^+\rangle_{AB} = \frac{1}{\sqrt{2}}(|00\rangle + |11\rangle)$. Alice holds systems A and C，Bob holds system B. **Note: Only quantum information is transferred, not the physical qubits.** 


<center><img src="figures/teleportation-fig-circuit.jpg" height="400" width="600"></center>
<div style="text-align:center">Figure 1: Quantum teleportation: Transferring quantum state $|\psi\rangle$ from Alice to receiver Bob. </div>


**Step I:** At the very beginning, the system state can be described as

$$
\lvert\varphi_{0}\rangle 
= \lvert\psi\rangle_{C}\otimes \lvert\Phi^+\rangle_{AB} 
= \frac{1}{\sqrt{2}}\big[\alpha\lvert0\rangle(\lvert00\rangle + \lvert11\rangle)+\beta\lvert1\rangle(\lvert00\rangle + \lvert11\rangle)\big],
\tag{1}
$$

where the quantum state Alice want to transmit is $|\psi\rangle_C = \alpha|0\rangle_C + \beta|1\rangle_C$ and the coefficients $\alpha, \beta \in \mathbb{C}$.

**Step II:** Alice applies a CNOT gate, and the resulting state is

$$
|\varphi_1\rangle  
= \frac{1}{\sqrt{2}}\big[\alpha\lvert0\rangle(\lvert00\rangle + \lvert11\rangle)+\beta\lvert1\rangle(\lvert10\rangle + \lvert01\rangle)\big],
\tag{2}
$$

**Step III:** Alice applies a Hadamard gate, and the system state becomes $|\varphi_2\rangle$ 

$$
|\varphi_2\rangle = \frac{1}{2}\big[\alpha(\lvert0\rangle + \lvert1\rangle)(\lvert00\rangle + \lvert11\rangle)+\beta(\lvert0\rangle - \lvert1\rangle)(\lvert10\rangle + \lvert01\rangle)\big],
\tag{3}
$$

The above state can be rearranged to

$$
\lvert\varphi_{2}\rangle = \frac{1}{2}\big[\lvert00\rangle(\alpha\lvert0\rangle + \beta\lvert1\rangle) + \lvert01\rangle(\alpha\lvert1\rangle + \beta\lvert0\rangle) + \lvert10\rangle(\alpha\lvert0\rangle - \beta\lvert1\rangle) + \lvert11\rangle(\alpha\lvert1\rangle - \beta\lvert0\rangle)\big].
\tag{4}
$$

**Step IV:** Alice measures both of her qubits in the computational basis $\{|00\rangle, |01\rangle, |10\rangle, |11\rangle\}$ and send the results $m_1m_2$ to Bob with a classical channel. There are 4 distinct possibilities: $m_1m_2 \in \{ 00, 01,10, 11\}$. Then, Bob implements certain operations correspondingly on his qubit based on the received messages.


- If the measurement result is $m_1m_2 = 00$, Bob's state will be $\alpha\lvert0\rangle + \beta\lvert1\rangle$, which is the state Alice want to transmit $\lvert\psi\rangle_C$. No operations are needed and the teleportation is finished.
- If the measurement result is $m_1m_2 = 01$, Bob's state will be $\alpha\lvert1\rangle + \beta\lvert0\rangle$. Bob needs to act the $X$ gate on his qubit.
- If the measurement result is $m_1m_2 = 10$, Bob's state will be $\alpha\lvert0\rangle - \beta\lvert1\rangle$. Bob needs to act the $Z$ gate on his qubit.
- If the measurement result is $m_1m_2 = 11$, Bob's state will be $\alpha\lvert1\rangle - \beta\lvert0\rangle$. Bob needs to act the $X$ gate followed by the $Z$ gate on his qubit.

In the next section, we will go through how to simulate the teleportation protocol with Paddle Quantum.

## Simulation with Paddle Quantum

First, we need to import all the dependencies:

In [1]:
import numpy as np
import paddle.fluid as fluid
from paddle_quantum.locc import LoccNet
from paddle.complex import matmul, trace
from paddle_quantum.utils import state_fidelity
from paddle_quantum.state import bell_state, isotropic_state, density_op_random

Initialize the quantum state, and define the quantum circuit and teleportation protocol.

In [2]:
class LOCC(LoccNet):
    def __init__(self):
        super(LOCC, self).__init__()
        
        # Initialize LOCCNet
        self.parties = list()
        
        # Add the first party Alice 
        # The first parameter 2 stands for how many qubits A holds
        # The second parameter records the name of this qubit holder
        self.add_new_party(2, party_name="Alice")
        
        # Add the second party Bob
        # The first parameter 1 stands for how many qubits B holds
        # The second parameter records the name of this qubit holder
        self.add_new_party(1, party_name="Bob")
        

        # Create a bell state
        _state = fluid.dygraph.to_variable(bell_state(2))
#         _state = fluid.dygraph.to_variable(isotropic_state(2, 0.8))

        
        # Generate random pure quantum states for teleportation
        random_state = density_op_random(n=1, real_or_complex=2, rank=1)
        self.state_C = fluid.dygraph.to_variable(random_state)
        
        # Initialize the system by distributing states between Alice and Bob
        # ("Alice", 0) refers to qubit C
        # ("Alice", 1) refers to qubit A
        # ("Bob", 0) refers to qubit B
#         print('Pre-shared entanglement state is:\n', _state.numpy())
        self.set_init_status(self.state_C, [("Alice", 0)])
        self.set_init_status(_state, [("Alice", 1), ("Bob", 0)])


    def teleportation(self):
        status = self.init_status
        
        # Create Alice's local operations 
        cirA = self.create_ansatz("Alice")
        cirA.cnot([0, 1])
        cirA.h(0)
        
        # Run circuit
        status = cirA.run(status)
        
        # Alice measures both of her qubits C and A
        status_A = self.measure(status, [("Alice", 0), ("Alice", 1)], ["00", "01", "10", "11"])
        
        # Record average fidelity 
        fid_list = []

        # Bob applies different gates on his qubits depending on the measurement result of Alice
        for i, s in enumerate(status_A):
            
            # Bob's circuit
            if status_A[i].measured_result == '00':
                
                # Create Bob's local operations 
                cirB = self.create_ansatz("Bob")
                
                # Run circuit
                status_B = cirB.run(s) 
                
                # Trace out the measured qubits C and A
                # Leaving out only Bob’s qubit B
                status_fin = self.partial_state(status_B, [("Bob", 0)])

                # Calculate the fidelity between the teleported state and the original state
                fid = state_fidelity(self.state_C.numpy(), status_fin.state.numpy())**2
                fid_list.append(fid * status_fin.prob.numpy()[0])
                
            elif status_A[i].measured_result == '01':
                cirB = self.create_ansatz("Bob")
                cirB.x(0)
                status_B = cirB.run(s)
                status_fin = self.partial_state(status_B, [("Bob", 0)])
                fid = state_fidelity(self.state_C.numpy(), status_fin.state.numpy())**2
                fid_list.append(fid * status_fin.prob.numpy()[0])
            elif status_A[i].measured_result == '10':
                cirB = self.create_ansatz("Bob")
                cirB.z(0)
                status_B = cirB.run(s)
                status_fin = self.partial_state(status_B, [("Bob", 0)])
                fid = state_fidelity(self.state_C.numpy(), status_fin.state.numpy())**2
                fid_list.append(fid * status_fin.prob.numpy()[0])
            elif status_A[i].measured_result == '11':
                cirB = self.create_ansatz("Bob")
                cirB.x(0)
                cirB.z(0)
                status_B = cirB.run(s)
                status_fin = self.partial_state(status_B, [("Bob", 0)])
                fid = state_fidelity(self.state_C.numpy(), status_fin.state.numpy())**2
                fid_list.append(fid * status_fin.prob.numpy()[0])
        fid_avg = sum(fid_list)
        
        return fid_avg

Then we randomly generate 200 pure quantum states and use state fidelity $F$ to benchmark the teleportation protocol, where

$$
F(\rho,\sigma) \equiv \text{tr}\big( \sqrt{\sqrt{\rho}\sigma \sqrt{\rho}} \big)^2.
\tag{5}
$$

In [3]:
with fluid.dygraph.guard():
    
    np.random.seed(999)     # Fix random seed
    num_state = 200         # Number of random states generated
    list_fid = []           # Record the fidelity
    
    # Start sampling
    for idx in range(num_state):
        list_fid.append(LOCC().teleportation())

    print('Teleportation_Fidelity_Avg:', np.around(sum(list_fid)/len(list_fid), 4), ', std=', np.std(list_fid))

Teleportation_Fidelity_Avg: 1.0 , std= 1.0610206920288784e-14


**Note:** We want to point out the validity of this protocol relies on the quality of the pre-shared entanglement. Readers can change the entangled state from `bell_state(2)` to `isotropic_state(2, p)` and see how quantum noises will influence the teleportation performance. Recall the definition of isotropic states,


$$
\rho_{\text{iso}}(p) = p\lvert\Phi^+\rangle \langle\Phi^+\rvert + (1-p)\frac{I}{4}, \quad p \in [0,1]
\tag{6}
$$

## Learning a teleportation protocol with LOCCNet


### Training the protocol 

A general LOCC protocol can be classified by the number of classical communication rounds $r$. The original teleportation protocol is a one-round ($r=1$) protocol. For simplicity, we also restrict the communication rounds to 1. Compare to the original protocol, we use parametrized quantum circuits (PQC) to replace the fixed gates $U\in\{X,Z\}$ Bob applied on his qubit with a general rotation gate $U_3$ on the Bloch sphere. 

$$
U_3(\theta, \phi, \varphi) =
\begin{bmatrix}
\cos(\frac{\theta}{2})           & -e^{i\varphi}\sin(\frac{\theta}{2})\\
e^{i\phi}\sin(\frac{\theta}{2})  & e^{i(\phi+\varphi)} \cos(\frac{\theta}{2})
\end{bmatrix}.
\tag{7}
$$

This would bring us a more powerful searching capability in finding practical LOCC protocols. Similarly, we change Alice's local operations to a more general PQC called the `universal_2_qubit_gate(theta, which_qubit=[0,1])` [4]. We summarize the workflow below: 

1. Alice applies a 2-qubit PQC on her qubits.
2. Alice measures both of her qubits in the computational basis and communicates with Bob through a classical channel.
3. There are 4 possible measurement results: $m_1m_2 \in \{00, 01, 10, 11\}$. Bob needs to act different operations corresponding to these measurement results. After Bob's local operations, the state on his qubit collapses to $\lvert \psi\rangle_{B}$. 
4. Calculate the overlap $O$ between $\lvert \psi\rangle_{B}$ and $\lvert\psi_C\rangle$ (pure states). LOCCNet framework only supports density matrix formulation and hence we have to rewrite them as $\rho_{B} = |\psi\rangle\langle\psi|_B$ and $\rho_{C} = |\psi\rangle\langle\psi|_C$. Then, $O = \text{Tr}(\rho_C\rho_{B})$. For pure states, this metric is simply the state fidelity.
5. Set the accumulated loss function over 4 possible measurement results as $L = \sum_{m_1m_2} \big(1-\text{Tr}(\rho_C\rho_{B})\big)$, and use gradient-based optimization methods to update circuit parameters and hence minimize the loss function.
6. Repeat steps 1-5 until the loss function converges to the global minimum.
7. Generate an ensemble of arbitrary states $\{\lvert\psi_C\rangle\}$, and benchmark the trained protocol with average state fidelity.


<center><img src="figures/teleportation-fig-LOCCNet.png" height="400" width="600"></center>
<div style="text-align:center">Figure 2: Learning a teleportation protocol with LOCCNet. </div>


**Note:** In order to make sure the parameters in parameterized quantum circuit is valid for all state after training, we set the training set as 4 linear independent states, which is $\{|0\rangle\langle 0|,|1\rangle\langle 1|,|+\rangle\langle +|,|+\rangle\langle +|_y\}$ or in the density matrix form:

$$
\rho_0 = \left[\begin{array}{ccc}
1 & 0\\
0 & 0
\end{array}\right], 
\rho_1 = \left[\begin{array}{ccc}
0 & 0\\
0 & 1
\end{array}\right], 
\rho_2 = \left[\begin{array}{ccc}
0.5 & 0.5\\
0.5 & 0.5
\end{array}\right], 
\rho_3 = \left[\begin{array}{ccc}
0.5 & -0.5 i\\
0.5i & 0.5
\end{array}\right]. 
\tag{8}
$$

Any single qubit state can be written as a combination of the above 4 linear independent states in $\mathcal{H}^{2\times 2}$.

In [ ]:
class LOCC_Train(LoccNet):
    def __init__(self):
        super(LOCC_Train, self).__init__()
        
        # Initialize LOCCNet
        self.parties = list()
        
        # Add the first party Alice 
        # The first parameter 2 stands for how many qubits A holds
        # The second parameter records the name of this qubit holder
        self.add_new_party(2, party_name="Alice")
        
        # Add the second party Bob
        # The first parameter 1 stands for how many qubits B holds
        # The second parameter records the name of this qubit holder
        self.add_new_party(1, party_name="Bob")
        

        # Create a bell state
        _state = fluid.dygraph.to_variable(bell_state(2))
#         _state = fluid.dygraph.to_variable(isotropic_state(2, 0.8))


        # Initialize Alice‘s parameters
        self.theta_A = self.create_parameter(shape=[15], attr=fluid.initializer.Uniform(low=0.0, high=2 * np.pi, 
                                                                                        seed=SEED), dtype="float64")
        # Initialize Bob's parameters
        self.theta_B = self.create_parameter(shape=[4, 3], attr=fluid.initializer.Uniform(low=0.0, high=2 * np.pi, 
                                                                                          seed=SEED), dtype="float64")

        # Training set: 4 linear independent states
        _state0 = fluid.dygraph.to_variable(np.array([[1, 0], [0, 0]], dtype=np.complex128))
        _state1 = fluid.dygraph.to_variable(np.array([[0, 0], [0, 1]], dtype=np.complex128))
        _state2 = fluid.dygraph.to_variable(np.array([[0.5, 0.5], [0.5, 0.5]], dtype=np.complex128))
        _state3 = fluid.dygraph.to_variable(np.array([[0.5, -0.5j], [0.5j, 0.5]], dtype=np.complex128))
        self.init_states = [_state0, _state1, _state2, _state3]

        # Initialize the system by distributing states between Alice and Bob
        self.set_init_status(_state, [("Alice", 1), ("Bob", 0)])
        self.set_init_status(_state0, [("Alice", 0)])

    def LOCCNet(self):
        
        # Define the training process
        loss = 0
        temp_state = self.init_status
        
        # Training
        for init_state in self.init_states:
            
            # Reset Alice's first qubit C to states in training set
            status = self.reset_state(temp_state, init_state, [("Alice", 0)])

            # Define Alice's local operations
            cirA = self.create_ansatz("Alice")
            cirA.universal_2_qubit_gate(self.theta_A, [0, 1])

            # Run circuit
            status = cirA.run(status)
            
             # Obtain 4 possible measurement results
            status_A = self.measure(status, [("Alice", 0), ("Alice", 1)], ["00", "01", "10", "11"])
            
            # Bob needs to apply different operation on his qubits, depending on the measurement results
            for i, s in enumerate(status_A):
                
                # Define Bob's local operations
                cirB = self.create_ansatz("Bob")
                
                # Apply a universal single qubit gate
                cirB.u3(*self.theta_B[i], 0)
                
                # Run circuit
                status_B = cirB.run(s)
                
                # Trace out the measured qubits C and A
                # Leaving out only Bob’s qubit B
                status_fin = self.partial_state(status_B, [("Bob", 0)])
                
                # Summing up the loss function for all possible measurement results
                loss += 1 - trace(matmul(init_state, status_fin.state)).real[0]
        
        return loss

    # Save the optimized parameters
    def save_module(self):
        theta = np.array([self.theta_A.numpy(), self.theta_B.numpy()])
        np.save('parameters/QT_LOCCNet', theta)

In [ ]:
ITR = 150   # Number of iterations
LR = 0.2    # Set up learning rate
SEED = 999  # Fix random seed for parameters in PQC

# Initialize the dynamic graph mode
with fluid.dygraph.guard():

    net = LOCC_Train()
    
    # Choose the Adam optimizer
    opt = fluid.optimizer.AdamOptimizer(learning_rate=LR, parameter_list= net.parameters())
    
    # Record loss
    loss_list = []
    
    # Optimization loop
    for itr in range(ITR):
        
        # Forward propagation to calculate loss function
        loss = net.LOCCNet()
        
        # Backpropagation
        loss.backward()
        opt.minimize(loss)
        
        # Clean gradients
        net.clear_gradients()
        
        # Record the learning process
        loss_list.append(loss.numpy()[0])
        if itr % 10 == 0:
            print("itr " + str(itr) + ":", loss.numpy()[0])
            
    # Save parameters
    net.save_module()

### Benchmark

If you don't want to spend time on training, you can **load the pre-trained circuit parameters** and directly test the performance.

In [4]:
class LOCC_Test(LoccNet):
    def __init__(self):
        super(LOCC_Test, self).__init__()
        
        self.parties = list()
        self.add_new_party(2, party_name="Alice")
        self.add_new_party(1, party_name="Bob")
        
        self.theta_A = fluid.dygraph.to_variable(para[0])
        self.theta_B = fluid.dygraph.to_variable(para[1])
        
        _state = fluid.dygraph.to_variable(bell_state(2))
        random_state = density_op_random(n=1)
        self._state0 = fluid.dygraph.to_variable(random_state)
        self.set_init_status(_state, [("Alice", 1), ("Bob", 0)])
        self.set_init_status(self._state0, [("Alice", 0)])
        

    def benchmark(self):
        input_state = self.init_status
        cirA = self.create_ansatz("Alice")
        cirA.universal_2_qubit_gate(self.theta_A, [0, 1])

        status = cirA.run(input_state)
        status_A = self.measure(status, [("Alice", 0), ("Alice", 1)], ["00", "01", "10", "11"])
        fid_list = []

        for i, s in enumerate(status_A):
            cirB = self.create_ansatz("Bob")
            cirB.u3(*self.theta_B[i], 0)
            status_B = cirB.run(s)
            status_fin = self.partial_state(status_B, [("Bob", 0)])
            fid = state_fidelity(self._state0.numpy(), status_fin.state.numpy())**2
            fid_list.append(fid*status_fin.prob.numpy()[0])
        fid_ave = sum(fid_list)
        
        return fid_ave

In [5]:
with fluid.dygraph.guard():

    # Load pre-trained circuit parameters
    para = np.load('parameters/QT_LOCCNet_Trained.npy', allow_pickle=True)
    np.random.seed(999)     # Fix random seed
    num_state = 200         # Number of random states generated
    list_fid = []           # Record the fidelity
    
    # Start sampling
    for idx in range(num_state):
        list_fid.append(LOCC_Test().benchmark())

    print('LOCCNet_Fidelity_avg:', np.around(sum(list_fid)/len(list_fid), 4), ', std=', np.std(list_fid))

LOCCNet_Fidelity_avg: 1.0 , std= 5.695904177817817e-07


## Conclusion

Based on LOCCNet we successfully learned a teleportation protocol with a noiseless pre-shared Bell state. The original teleportation protocol was designed to transfer a single-qubit quantum state. It is not clear how it can be generalize to the multi-qubit case. By comparison, LOCCNet provides the possibility of training a teleportation protocol for multi-qubit quantum states. Also, it will be an interesting question to ask how robust LOCCNet will  be against various noises. On the other hand, the teleportation protocol could be viewed as a special case of simulation the identity channel $\mathcal{E}_I$ where Alice sends $\psi$ and Bob receives $\mathcal{E}_I(\psi)$. This idea can be extended to simulate many other channels including the depolarizing channel $\mathcal{E}_{D}$.

---

## References

[1] Bennett, Charles H., et al. "Teleporting an unknown quantum state via dual classical and Einstein-Podolsky-Rosen channels." [Physical Review Letters 70.13 (1993): 1895.](https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.70.1895)

[2] Boschi, Danilo, et al. "Experimental realization of teleporting an unknown pure quantum state via dual classical and Einstein-Podolsky-Rosen channels." [Physical Review Letters 80.6 (1998): 1121.](https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.80.1121)

[3] Bouwmeester, Dik, et al. "Experimental quantum teleportation." [Nature 390.6660 (1997): 575-579.](https://www.nature.com/articles/37539)

[4] Vidal, Guifre, and Christopher M. Dawson. "Universal quantum circuit for two-qubit transformations with three controlled-NOT gates." [Physical Review A 69.1 (2004): 010301.](https://journals.aps.org/pra/abstract/10.1103/PhysRevA.69.010301)
